In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
demand_set=pd.read_csv('Demand_History.csv')

In [3]:
x = [[2010],[2011],[2012],[2013],[2014],[2015],[2016],[2017],[2018]]

In [4]:
x=np.array(x)

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [6]:
poly_reg3=PolynomialFeatures(degree=4)
x_poly_3=poly_reg3.fit_transform(x)

In [7]:
ncol1=[]
ncol2=[]
lin_reg_3=LinearRegression()
for i in range(4096):
    y = demand_set.iloc[i,3:].values
    lin_reg_3.fit(x_poly_3,y)
    ans1=lin_reg_3.predict(poly_reg3.transform([[2019]]))
    ncol1.append(ans1[0])
demand_set['2019']=ncol1

In [8]:
x = [[2010],[2011],[2012],[2013],[2014],[2015],[2016],[2017],[2018],[2019]]
x_poly_3=poly_reg3.fit_transform(x)
for i in range(4096):
    y = demand_set.iloc[i,3:].values
    lin_reg_3.fit(x_poly_3,y)
    ans2=lin_reg_3.predict(poly_reg3.transform([[2020]]))
    ncol2.append(ans2[0])
demand_set['2020']=ncol2

In [9]:
demand_set

,demand_point_index,x_coordinate,y_coordinate,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0,0.5,0.5,0.352242,0.667932,0.958593,2.911901,4.338274,6.561995,8.454417,10.595324,13.119572,15.087002,16.921831
1,1,1.5,0.5,0.325940,0.591964,0.862652,2.589068,4.196034,5.745551,8.753195,11.126995,12.020091,13.445920,13.816723
2,2,2.5,0.5,0.373752,0.591890,0.969733,2.641432,3.541772,5.469161,8.414627,10.115336,14.018254,17.502170,21.636805
3,3,3.5,0.5,0.420686,0.584055,0.906547,2.378577,3.888121,5.846089,9.083868,12.424885,15.012302,18.672595,22.176754
4,4,4.5,0.5,0.475621,0.647940,0.981544,2.665400,4.218711,6.776609,8.851107,11.731131,16.355563,20.527613,25.625417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4091,4091,59.5,63.5,0.171015,0.334565,0.556055,1.373291,1.837586,2.517146,3.352280,4.149888,5.426193,6.540874,7.851604
4092,4092,60.5,63.5,0.041716,0.061741,0.131291,0.386540,0.755846,0.941116,1.107797,1.309479,2.057450,2.460323,3.066954
4093,4093,61.5,63.5,0.100895,0.180352,0.296299,0.705373,1.300220,1.608609,1.822806,2.333681,3.218519,3.786670,4.544978
4094,4094,62.5,63.5,0.155353,0.290825,0.557803,1.516066,2.399426,2.719197,4.494515,6.096858,6.262574,7.280598,7.744934


In [10]:
demand_set.to_csv('Demand_Prediction.csv')

In [11]:
from pulp import *

In [12]:
n_supply = 100   #number of supply points
n_demand = 4096  #number of demand points

In [13]:
#Importing csv files 
demand_set = pd.read_csv('Demand_Prediction.csv') 
supply_set = pd.read_csv('exisiting_EV_infrastructure_2018.csv')
dist_mat=pd.read_csv('Distance_Matrix.csv')

In [14]:
dist_mat=dist_mat.to_numpy()

In [15]:
#Smax 2018
col=[]
for i in range(supply_set.shape[0]):
    ans=supply_set.iloc[i:i+1,4].values*200+supply_set.iloc[i:i+1,5].values*400
    col.append(int(ans))
col=np.array(col)
supply_set['Smax18']=col

In [16]:
#Demand values of 2019
demands19=np.array(demand_set['2019'])

#Initialize the PuLP Model
model = LpProblem("Cost-Optimization-Model", LpMinimize)

#Create variable index ids
index_numbers_x = [str(i) for i in range(n_supply*n_demand)]
index_numbers_y = [str(i) for i in range(n_supply)]
index_numbers_x.sort()
index_numbers_y.sort()

#Declare X variable (DS)
DV_variables = LpVariable.matrix("X", index_numbers_x, cat = "Continuous", lowBound= 0 )
dsij_mat19 = np.array(DV_variables).reshape(n_supply,n_demand)

#Empty parking slots
Smax19=[]
park_slot=supply_set['total_parking_slots']-(supply_set['existing_num_SCS']+supply_set['existing_num_FCS'])
park_slot.to_numpy()

#Declare Y variable (Smax)
for i in range(n_supply):
    lb=int(supply_set.iloc[i:i+1,6].values)
    ub=lb+(park_slot[i]*400)
    temp=LpVariable('Y_{}'.format(index_numbers_y[i]),cat='Integer',lowBound=lb,upBound=ub)
    Smax19.append(temp)
Smax19=np.array(Smax19)

#Customer Dissatisfaction cost 
cost_eq1 = lpSum(dsij_mat19*dist_mat)

S_var=[]
F_var=[]
index_numbers_c = [str(i) for i in range(n_supply)]
index_numbers_c.sort()

#Declare S variable (SCS)
for i in range(n_supply):
    lb=int(supply_set.iloc[i:i+1,4].values)
    temp=LpVariable('S_{}'.format(index_numbers_c[i]),cat='Integer', lowBound=lb)
    S_var.append(temp)
S_var=np.array(S_var)

#Declare F variable (FCS)
for i in range(n_supply):
    lb=int(supply_set.iloc[i:i+1,5].values)
    temp=LpVariable('F_{}'.format(index_numbers_c[i]),cat='Integer',lowBound=lb)
    F_var.append(temp)
F_var=np.array(F_var)

#Infrastructure cost
cost_eq2=lpSum(S_var)+1.5*lpSum(F_var)

#Final cost
cost_eq3=cost_eq1+(600*cost_eq2)

#Adding the objective equation to the model (minimize)
model += cost_eq3

#Supply Constraint
#Column summation must be less than or equal to Smax 
for i in range(n_supply):
    model += lpSum(dsij_mat19[i][j] for j in range(n_demand)) <= Smax19[i]-0.001 , "Supply Constraints " + str(i)

# Demand Constraints
#Row summation must be equal to demand forecast
for j in range(n_demand):
    model += lpSum(dsij_mat19[i][j] for i in range(n_supply)) >= demands19[j] , "Demand Constraints " + str(j)

#Smax constraint
for i in range(n_supply):
    constraint = (S_var[i]*200)+(F_var[i]*400) == Smax19[i]
    model += constraint

In [17]:
model.solve(PULP_CBC_CMD())
status =  LpStatus[model.status]
print(status)

Optimal


In [18]:
print("Total Cost:", model.objective.value())

Total Cost: 2586142.012064943


In [19]:
allVar=[]
for v in model.variables():
    k=v.value()
    allVar.append(k)

p=0
F_var=[]
S_var=[]
for i in range(n_supply):
    F_var.append(int(allVar[p]))
    p+=1
for i in range(n_supply):
    S_var.append(int(allVar[p]))
    p+=1

dsij_mat19=np.zeros((100,4096))
    
for i in range(n_supply):
    for j in range(n_demand):
        dsij_mat19[i][j]=allVar[p]
        p+=1

Smax19=[]
for i in range(n_supply):
    Smax19.append(int(allVar[p]))
    p+=1
Smax19=np.array(Smax19)

dsij_mat19=pd.DataFrame(dsij_mat19)
dsij_mat19.to_csv('Dsij_19.csv') #Export DSij matrix

In [20]:
#Checking the Demand constraint (row summation)
print(sum(dsij_mat19[0]))
#Comparing with demand value
print(demands19[0])

15.087002
15.087002202868462


In [21]:
#Checking the Supply constraint (column summation)
print(sum(dsij_mat19.iloc[0,:].values))
#Comparing with supply value
print(Smax19[0])

7799.9989788
7800


In [22]:
supply_set['Smax19']=Smax19
supply_set['SCS_19']=S_var
supply_set['FCS_19']=F_var

In [23]:
supply_set2=pd.DataFrame()
supply_set2['supply_point_index']=supply_set['supply_point_index']
supply_set2['x_coordinate']=supply_set['x_coordinate']
supply_set2['y_coordinate']=supply_set['y_coordinate']
supply_set2['total_parking_slots']=supply_set['total_parking_slots']
supply_set2['existing_num_SCS']=supply_set['SCS_19']
supply_set2['existing_num_FCS']=supply_set['FCS_19']
supply_set2.to_csv('exisiting_EV_infrastructure_2019.csv',index=0) #Export new infrastructure

In [24]:
supply_set2

,supply_point_index,x_coordinate,y_coordinate,total_parking_slots,existing_num_SCS,existing_num_FCS
0,0,50.163110,19.412014,23,5,17
1,1,37.336451,58.119225,27,4,7
2,2,46.709232,57.525650,31,6,14
3,3,30.528626,55.379835,26,5,5
4,4,51.521781,35.116755,32,11,6
...,...,...,...,...,...,...
95,95,45.471204,20.999414,24,3,9
96,96,30.318396,33.388335,32,5,22
97,97,36.218839,22.235766,32,4,25
98,98,42.936915,38.122442,28,7,5


In [25]:
supply_set = pd.read_csv('exisiting_EV_infrastructure_2019.csv')

In [26]:
supply_set

,supply_point_index,x_coordinate,y_coordinate,total_parking_slots,existing_num_SCS,existing_num_FCS
0,0,50.163110,19.412014,23,5,17
1,1,37.336451,58.119225,27,4,7
2,2,46.709232,57.525650,31,6,14
3,3,30.528626,55.379835,26,5,5
4,4,51.521781,35.116755,32,11,6
...,...,...,...,...,...,...
95,95,45.471204,20.999414,24,3,9
96,96,30.318396,33.388335,32,5,22
97,97,36.218839,22.235766,32,4,25
98,98,42.936915,38.122442,28,7,5


In [27]:
col=[]
for i in range(supply_set.shape[0]):
    ans=supply_set.iloc[i:i+1,4].values*200+supply_set.iloc[i:i+1,5].values*400
    col.append(int(ans))
col=np.array(col)
supply_set['Smax19']=col

In [28]:
supply_set

,supply_point_index,x_coordinate,y_coordinate,total_parking_slots,existing_num_SCS,existing_num_FCS,Smax19
0,0,50.163110,19.412014,23,5,17,7800
1,1,37.336451,58.119225,27,4,7,3600
2,2,46.709232,57.525650,31,6,14,6800
3,3,30.528626,55.379835,26,5,5,3000
4,4,51.521781,35.116755,32,11,6,4600
...,...,...,...,...,...,...,...
95,95,45.471204,20.999414,24,3,9,4200
96,96,30.318396,33.388335,32,5,22,9800
97,97,36.218839,22.235766,32,4,25,10800
98,98,42.936915,38.122442,28,7,5,3400


In [29]:
#Demand values of 2020
demands20=np.array(demand_set['2020'])

In [30]:
model = LpProblem("Cost-Optimization-Model", LpMinimize)

In [31]:
index_numbers_x = [str(i) for i in range(n_supply*n_demand)]
index_numbers_y = [str(i) for i in range(n_supply)]
index_numbers_x.sort()
index_numbers_y.sort()

In [32]:
DV_variables = LpVariable.matrix("X", index_numbers_x, cat = "Continuous", lowBound= 0 )
dsij_mat20 = np.array(DV_variables).reshape(n_supply,n_demand)

In [33]:
Smax20=[]
park_slot=supply_set['total_parking_slots']-(supply_set['existing_num_SCS']+supply_set['existing_num_FCS'])
park_slot.to_numpy()
for i in range(n_supply):
    lb=int(supply_set.iloc[i:i+1,6].values)
    ub=lb+(park_slot[i]*400)
    temp=LpVariable('Y_{}'.format(index_numbers_y[i]),cat='Integer',lowBound=lb,upBound=ub)
    Smax20.append(temp)
Smax20=np.array(Smax20)

In [34]:
cost_eq1 = lpSum(dsij_mat20*dist_mat)

In [35]:
S_var=[]
F_var=[]

index_numbers_c = [str(i) for i in range(n_supply)]
index_numbers_c.sort()

for i in range(n_supply):
    lb=int(supply_set.iloc[i:i+1,4].values)
    temp=LpVariable('S_{}'.format(index_numbers_c[i]),cat='Integer', lowBound=lb)
    S_var.append(temp)
S_var=np.array(S_var)

for i in range(n_supply):
    lb=int(supply_set.iloc[i:i+1,5].values)
    temp=LpVariable('F_{}'.format(index_numbers_c[i]),cat='Integer',lowBound=lb)
    F_var.append(temp)
F_var=np.array(F_var)

In [36]:
cost_eq2=lpSum(S_var)+1.5*lpSum(F_var)

In [37]:
cost_eq3=cost_eq1+(600*cost_eq2)

In [38]:
model += cost_eq3

In [39]:
#Supply Constraint
#Column summation must be less than or equal to Smax 
for i in range(n_supply):
    model += lpSum(dsij_mat20[i][j] for j in range(n_demand)) <= Smax20[i]-0.001 , "Supply Constraints " + str(i)

In [40]:
# Demand Constraints
#Row summation must be equal to demand forecast
for j in range(n_demand):
    model += lpSum(dsij_mat20[i][j] for i in range(n_supply)) >= demands20[j] , "Demand Constraints " + str(j)

In [41]:
#Supply Constraint
for i in range(n_supply):
    constraint = (S_var[i]*200)+(F_var[i]*400) >= Smax20[i]
    model += constraint

In [42]:
model.solve(PULP_CBC_CMD())
status =  LpStatus[model.status]
print(status)

Optimal


In [43]:
print("Total Cost:", model.objective.value())

Total Cost: 2882828.1292252108


In [44]:
allVar=[]
for v in model.variables():
    k=v.value()
    allVar.append(k)
#allVar

In [45]:
p=0
F_var=[]
S_var=[]
for i in range(n_supply):
    F_var.append(int(allVar[p]))
    p+=1
for i in range(n_supply):
    S_var.append(int(allVar[p]))
    p+=1
#print(F_var)
#print(S_var)

In [46]:
dsij_mat20=np.zeros((100,4096))
    
for i in range(n_supply):
    for j in range(n_demand):
        dsij_mat20[i][j]=allVar[p]
        p+=1
#dsij_mat20

In [47]:
Smax20=[]
for i in range(n_supply):
    Smax20.append(int(allVar[p]))
    p+=1
Smax20=np.array(Smax20)
#Smax20

In [48]:
dsij_mat20=pd.DataFrame(dsij_mat20)
dsij_mat20.to_csv('Dsij_20.csv')

In [49]:
#Checking the Demand constraint (row summation)
print(sum(dsij_mat20[0]))
#Comparing with demand value
print(demands20[0])

16.921831
16.92183116823435


In [50]:
#Checking the Supply constraint (column summation)
print(sum(dsij_mat20.iloc[0,:].values))
#Comparing with supply value
print(Smax20[0])

8199.9990084
8200


In [51]:
supply_set['Smax20']=Smax20
#supply_set

In [52]:
supply_set['SCS_20']=S_var
supply_set['FCS_20']=F_var


In [53]:
supply_set2=pd.DataFrame()
supply_set2['supply_point_index']=supply_set['supply_point_index']
supply_set2['x_coordinate']=supply_set['x_coordinate']
supply_set2['y_coordinate']=supply_set['y_coordinate']
supply_set2['total_parking_slots']=supply_set['total_parking_slots']
supply_set2['existing_num_SCS']=supply_set['SCS_20']
supply_set2['existing_num_FCS']=supply_set['FCS_20']
supply_set2.to_csv('exisiting_EV_infrastructure_2020.csv',index=0) #Export new infrastructure

In [54]:
dsij19=pd.read_csv('Dsij_19.csv',index_col=0)
infra19=pd.read_csv('exisiting_EV_infrastructure_2019.csv')

In [55]:
dsij19=dsij19.to_numpy()

In [56]:
sub=pd.DataFrame(columns=['year','data_type','demand_point_index','supply_point_index','value'])

In [57]:
for i in range(100):
    sub=sub.append({'year':'2019','data_type':'SCS','supply_point_index':'{}'.format(int(infra19.iloc[i:i+1,0].values)),'value':'{}'.format(int(infra19.iloc[i:i+1,-2].values))},ignore_index=True)
for i in range(100):
    sub=sub.append({'year':'2019','data_type':'FCS','supply_point_index':'{}'.format(int(infra19.iloc[i:i+1,0].values)),'value':'{}'.format(int(infra19.iloc[i:i+1,-1].values))},ignore_index=True)

In [58]:
sub

,year,data_type,demand_point_index,supply_point_index,value
0,2019,SCS,NaN,0,5
1,2019,SCS,NaN,1,4
2,2019,SCS,NaN,2,6
3,2019,SCS,NaN,3,5
4,2019,SCS,NaN,4,11
...,...,...,...,...,...
195,2019,FCS,NaN,95,9
196,2019,FCS,NaN,96,22
197,2019,FCS,NaN,97,25
198,2019,FCS,NaN,98,5


In [59]:
dsij19=pd.DataFrame(dsij19.reshape(409600,1,order='F'))

In [60]:
lstd=[]
for i in range(4096):
    for j in range(100):
        lstd.append(i)
lstd=pd.DataFrame(lstd)    

In [61]:
lsts=[]
for i in range(4096):
    for j in range(100):
        lsts.append(j)
lsts=pd.DataFrame(lsts)

In [62]:
sub=pd.concat([sub,lstd,lsts,dsij19],axis=1)
sub

,year,data_type,demand_point_index,supply_point_index,value,0,0,0
0,2019,SCS,NaN,0,5,0,0,0.0
1,2019,SCS,NaN,1,4,0,1,0.0
2,2019,SCS,NaN,2,6,0,2,0.0
3,2019,SCS,NaN,3,5,0,3,0.0
4,2019,SCS,NaN,4,11,0,4,0.0
...,...,...,...,...,...,...,...,...
409595,NaN,NaN,NaN,NaN,NaN,4095,95,0.0
409596,NaN,NaN,NaN,NaN,NaN,4095,96,0.0
409597,NaN,NaN,NaN,NaN,NaN,4095,97,0.0
409598,NaN,NaN,NaN,NaN,NaN,4095,98,0.0


In [63]:
sub.to_csv('draft19.csv')

In [64]:
dsij20=pd.read_csv('Dsij_20.csv',index_col=0)
infra20=pd.read_csv('exisiting_EV_infrastructure_2020.csv')

In [65]:
dsij20=dsij20.to_numpy()

In [66]:
sub=pd.DataFrame(columns=['year','data_type','demand_point_index','supply_point_index','value'])

In [67]:
for i in range(100):
    sub=sub.append({'year':'2020','data_type':'SCS','supply_point_index':'{}'.format(int(infra20.iloc[i:i+1,0].values)),'value':'{}'.format(int(infra20.iloc[i:i+1,-2].values))},ignore_index=True)
for i in range(100):
    sub=sub.append({'year':'2020','data_type':'FCS','supply_point_index':'{}'.format(int(infra20.iloc[i:i+1,0].values)),'value':'{}'.format(int(infra20.iloc[i:i+1,-1].values))},ignore_index=True)

In [68]:
sub

,year,data_type,demand_point_index,supply_point_index,value
0,2020,SCS,NaN,0,5
1,2020,SCS,NaN,1,4
2,2020,SCS,NaN,2,6
3,2020,SCS,NaN,3,5
4,2020,SCS,NaN,4,11
...,...,...,...,...,...
195,2020,FCS,NaN,95,18
196,2020,FCS,NaN,96,25
197,2020,FCS,NaN,97,28
198,2020,FCS,NaN,98,5


In [69]:
dsij20=pd.DataFrame(dsij20.reshape(409600,1,order='F'))

In [70]:
lstd=[]
for i in range(4096):
    for j in range(100):
        lstd.append(i)
lstd=pd.DataFrame(lstd)    

In [71]:
lsts=[]
for i in range(4096):
    for j in range(100):
        lsts.append(j)
lsts=pd.DataFrame(lsts)

In [72]:
sub=pd.concat([sub,lstd,lsts,dsij20],axis=1)
sub

,year,data_type,demand_point_index,supply_point_index,value,0,0,0
0,2020,SCS,NaN,0,5,0,0,0.0
1,2020,SCS,NaN,1,4,0,1,0.0
2,2020,SCS,NaN,2,6,0,2,0.0
3,2020,SCS,NaN,3,5,0,3,0.0
4,2020,SCS,NaN,4,11,0,4,0.0
...,...,...,...,...,...,...,...,...
409595,NaN,NaN,NaN,NaN,NaN,4095,95,0.0
409596,NaN,NaN,NaN,NaN,NaN,4095,96,0.0
409597,NaN,NaN,NaN,NaN,NaN,4095,97,0.0
409598,NaN,NaN,NaN,NaN,NaN,4095,98,0.0


In [73]:
sub.to_csv('draft20.csv')